<a href="https://colab.research.google.com/github/MikJak75/ResearchCopies/blob/main/Optuna_Test_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [43]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
import os

import optuna
from optuna.trial import TrialState

import torch
import numpy as np

from torch import optim, nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset

#DEVICE = torch.device("cpu")

In [45]:

#class IrisDataset(Dataset):
#    def __init__(self, path, delimiter=','):
#        self.__data = np.genfromtxt(path, delimiter=delimiter).astype(np.float32)
#
#    def __getitem__(self, index):
#        instance = self.__data[index,:]
#        data = torch.from_numpy(instance[:-1])
#        label = torch.from_numpy(np.array(instance[-1]).astype(int))
#
#        return data, label
#
#    def __len__(self):
#        return self.__data.shape[0]

class IrisDataset_Train(Dataset):
    def __init__(self, path, delimiter=','):
        self.__data = np.genfromtxt(path, delimiter=delimiter).astype(np.float32)
        self.__data = self.__data[0:100]

    def __getitem__(self, index):
        instance = self.__data[index,:]
        data = torch.from_numpy(instance[:-1])
        label = torch.from_numpy(np.array(instance[-1]).astype(int))

        return data, label

    def __len__(self):
        return self.__data.shape[0]

class IrisDataset_Test(Dataset):
    def __init__(self, path, delimiter=','):
        self.__data = np.genfromtxt(path, delimiter=delimiter).astype(np.float32)
        self.__data = self.__data[100:]

    def __getitem__(self, index):
        instance = self.__data[index,:]
        data = torch.from_numpy(instance[:-1])
        label = torch.from_numpy(np.array(instance[-1]).astype(int))

        return data, label

    def __len__(self):
        return self.__data.shape[0]


In [63]:

def define_model(trial):
  layer1_out_size = trial.suggest_int("size_l1{}".format(i), 4, 10)

  self.fc1 = nn.Linear(4, 10)
  self.fc2 = nn.Linear(layer1_out_size, 10)
  self.fc3 = nn.Linear(10, 3)

  x = F.relu(self.fc1(x))
  x = F.relu(self.fc2(x))
  return F.log_softmax(self.fc3(x), dim=1)
   

In [64]:
 
def get_data():
  train_loader = DataLoader(dataset=IrisDataset_Train('/content/gdrive/My Drive/iris.data'),
                        batch_size=10,
                        shuffle=True)
  valid_loader = DataLoader(dataset=IrisDataset_Valid('/content/gdrive/My Drive/iris.data'),
                        batch_size=10,
                        shuffle=True)

  return train_loader, valid_loader


In [61]:

EPOCHS = 10
  

def objective(trial):

  model = define_model(trial)
  model.train()

  #ptimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
  #lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
  #optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

  train_loader, valid_loader = get_data()



  #model.train()
  #for epoch in range(EPOCHS):
  #  #model.train()
  #  for batch_idx, (data, target) in enumerate(train_loader):
  #      # Limiting training data for faster epochs.
  #      if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
  #          break
#
  #      data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
#
  #      optimizer.zero_grad()
  #      output = model(data)
  #      loss = F.nll_loss(output, target)
  #      loss.backward()
  #      optimizer.step()


  epochs = 50
  model = define_model(trial)
  criterion = nn.NLLLoss()
  #optimizer = optim.SGD(model.parameters(), lr=0.01)

  optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
  #lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
  optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=0.01)

  for epoch in range(epochs):
      #running_loss = 0
      for instances, labels in dataloader:
          optimizer.zero_grad()

          output = model(instances)
          loss = criterion(output, labels)
          loss.backward()
          optimizer.step()

      model.eval()
      correct = 0
      with torch.no_grad():
          for batch_idx, (data, target) in enumerate(valid_loader):
              # Limiting validation data.
              #if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
              #    break
              data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
              output = model(data)
              # Get the index of the max log-probability.
              pred = output.argmax(dim=1, keepdim=True)
              correct += pred.eq(target.view_as(pred)).sum().item()

      accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)

      trial.report(accuracy, epoch)

      if trial.should_prune():
        raise optuna.exceptions.TrialPruned();

      return accuracy

#  instance = instances[0].view(1, 4)
#  label = labels[0].view(1, 1)
#  print(torch.exp(model(instance)), label)


In [65]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-06-30 19:55:42,580] A new study created in memory with name: no-name-aa2987b7-506d-4b0b-9e4f-58e77da82af7
[W 2022-06-30 19:55:42,585] Trial 0 failed because of the following error: NameError("name 'i' is not defined")
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-61-ef9e4a42c9e0>", line 7, in objective
    model = define_model(trial)
  File "<ipython-input-63-01e3a5adc70b>", line 3, in define_model
    layer1_out_size = trial.suggest_int("size_l1{}".format(i), 4, 10)
NameError: name 'i' is not defined


NameError: ignored

In [ ]:
for instances, labels in dataloader:
  model.eval()
  print(model(instances), labels)

In [ ]:
0.27896711230278015